In [1]:
import utils
import pandas as pd
from prediction import analyze_data

In [2]:
device = utils.getting_device()

train_df =pd.read_csv('data/train_30303.csv')
test_df = pd.read_csv('data/test_17669.csv')
print(train_df.shape, test_df.shape)

device: cuda
(30303, 4) (17669, 4)


## Base: BERT, GPT, CodeBERT, RoBERTa

In [3]:
model = 'bert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/bert_30303.pt
Model vertified: BERT


Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting extract embeddings......
Time elapsed: 3140.53 seconds, Data processed:10000
Time elapsed: 3910.11 seconds, Data processed:20000
Time elapsed: 8006.41 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/bert_17669.pt
Model vertified: BERT


Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting extract embeddings......
Time elapsed: 9117.45 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.48 seconds, Data predicted: 0
Time elapsed: 238.36 seconds, Data predicted: 5000
Time elapsed: 239.84 seconds, Data predicted: 10000
Time elapsed: 238.78 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 52.06%
Accuracy of Programming Task prediction: 16.10%


In [7]:
model = 'gpt'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/gpt_30303.pt
Model vertified: GPT
Starting extract embeddings......
Time elapsed: 3979.21 seconds, Data processed:10000
Time elapsed: 4906.47 seconds, Data processed:20000
Time elapsed: 5774.42 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/gpt_17669.pt
Model vertified: GPT
Starting extract embeddings......
Time elapsed: 5598.58 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.63 seconds, Data predicted: 0
Time elapsed: 242.26 seconds, Data predicted: 5000
Time elapsed: 244.55 seconds, Data predicted: 10000
Time elapsed: 241.90 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 39.60%
Accuracy of Programming Task prediction: 6.54%


In [3]:
model = 'codegpt'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/codegpt_30303.pt
Model vertified: CodeGPT


Some weights of the model checkpoint at AISE-TUDelft/CodeGPT-Multilingual were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting extract embeddings......
Time elapsed: 4693.29 seconds, Data processed:10000
Time elapsed: 4524.14 seconds, Data processed:20000
Time elapsed: 4881.01 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/codegpt_17669.pt
Model vertified: CodeGPT


Some weights of the model checkpoint at AISE-TUDelft/CodeGPT-Multilingual were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Starting extract embeddings......
Time elapsed: 12108.80 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.70 seconds, Data predicted: 0
Time elapsed: 222.55 seconds, Data predicted: 5000
Time elapsed: 227.07 seconds, Data predicted: 10000
Time elapsed: 228.05 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 60.65%
Accuracy of Programming Task prediction: 17.17%


In [4]:
model = 'codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/codebert_30303.pt
Model vertified: CodeBERT
Starting extract embeddings......
Time elapsed: 1434.52 seconds, Data processed:10000
Time elapsed: 1353.40 seconds, Data processed:20000
Time elapsed: 1432.11 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/codebert_17669.pt
Model vertified: CodeBERT
Starting extract embeddings......
Time elapsed: 1442.99 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.58 seconds, Data predicted: 0
Time elapsed: 153.82 seconds, Data predicted: 5000
Time elapsed: 154.34 seconds, Data predicted: 10000
Time elapsed: 155.78 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 51.37%
Accuracy of Programming Task prediction: 10.2

In [3]:
model = 'roberta'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/roberta_30303.pt
Model vertified: RoBERTa


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting extract embeddings......
Time elapsed: 2629.01 seconds, Data processed:10000
Time elapsed: 2629.72 seconds, Data processed:20000
Time elapsed: 4106.97 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/roberta_17669.pt
Model vertified: RoBERTa


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting extract embeddings......
Time elapsed: 5309.72 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.51 seconds, Data predicted: 0
Time elapsed: 215.38 seconds, Data predicted: 5000
Time elapsed: 215.23 seconds, Data predicted: 10000
Time elapsed: 217.45 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 51.14%
Accuracy of Programming Task prediction: 13.00%


## Training

In [4]:
model = 'epoch4_roberta'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch4_roberta_30303.pt
Model vertified: epoch4_roberta
Starting extract embeddings......
Time elapsed: 2985.74 seconds, Data processed:10000
Time elapsed: 1970.07 seconds, Data processed:20000
Time elapsed: 2473.00 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch4_roberta_17669.pt
Model vertified: epoch4_roberta
Starting extract embeddings......
Time elapsed: 4002.98 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.61 seconds, Data predicted: 0
Time elapsed: 222.42 seconds, Data predicted: 5000
Time elapsed: 222.10 seconds, Data predicted: 10000
Time elapsed: 225.15 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 59.35%
Accuracy of Programmi

In [7]:
model = 'epoch1_codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch1_codebert_30303.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 2384.81 seconds, Data processed:10000
Time elapsed: 2737.07 seconds, Data processed:20000
Time elapsed: 3175.02 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch1_codebert_17669.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 3160.35 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.46 seconds, Data predicted: 0
Time elapsed: 217.80 seconds, Data predicted: 5000
Time elapsed: 218.54 seconds, Data predicted: 10000
Time elapsed: 223.19 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 60.71%
Accuracy of Programming T

In [6]:
model = 'epoch2_codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch2_codebert_30303.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 2850.23 seconds, Data processed:10000
Time elapsed: 2774.67 seconds, Data processed:20000
Time elapsed: 3174.80 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch2_codebert_17669.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 3161.32 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.43 seconds, Data predicted: 0
Time elapsed: 223.19 seconds, Data predicted: 5000
Time elapsed: 224.69 seconds, Data predicted: 10000
Time elapsed: 223.41 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 61.06%
Accuracy of Programming T

In [5]:
model = 'epoch3_codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch3_codebert_30303.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 1473.62 seconds, Data processed:10000
Time elapsed: 2588.61 seconds, Data processed:20000
Time elapsed: 3137.25 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch3_codebert_17669.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 3180.45 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.43 seconds, Data predicted: 0
Time elapsed: 219.28 seconds, Data predicted: 5000
Time elapsed: 222.58 seconds, Data predicted: 10000
Time elapsed: 221.29 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 62.93%
Accuracy of Programming T

In [4]:
model = 'epoch4_codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch4_codebert_30303.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 2791.22 seconds, Data processed:10000
Time elapsed: 2709.19 seconds, Data processed:20000
Time elapsed: 1774.08 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch4_codebert_17669.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 1630.40 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.32 seconds, Data predicted: 0
Time elapsed: 156.59 seconds, Data predicted: 5000
Time elapsed: 154.25 seconds, Data predicted: 10000
Time elapsed: 152.78 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 63.73%
Accuracy of Programming T

In [3]:
model = 'epoch5_codebert'

analyze_data(model, train_df, test_df, device, mode = 'emb')

out/epoch5_codebert_30303.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 2883.59 seconds, Data processed:10000
Time elapsed: 2742.88 seconds, Data processed:20000
Time elapsed: 3129.76 seconds, Data processed:30000
End of extracting...Number of record: torch.Size([30303, 768])
out/epoch5_codebert_17669.pt
Model vertified: saved model
Starting extract embeddings......
Time elapsed: 3131.15 seconds, Data processed:10000
End of extracting...Number of record: torch.Size([17669, 768])
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Original train count: 30303, test count: 17669.
After mapping: train count: 30303, test count: 17669.
Time elapsed: 0.77 seconds, Data predicted: 0
Time elapsed: 216.71 seconds, Data predicted: 5000
Time elapsed: 216.09 seconds, Data predicted: 10000
Time elapsed: 218.19 seconds, Data predicted: 15000
Accuracy of Programming Language prediction: 63.26%
Accuracy of Programming T